# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9ac0fd6fd0>
├── 'a' --> tensor([[ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180]])
└── 'x' --> <FastTreeValue 0x7f9ac0fd6fa0>
    └── 'c' --> tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                        [-1.0374,  0.9894,  0.7427, -0.7533],
                        [-1.7383,  0.4137,  0.2345,  0.6079]])

In [4]:
t.a

tensor([[ 0.9751,  0.0312, -0.8180],
        [-0.2948,  0.8449,  0.5180]])

In [5]:
%timeit t.a

75.8 ns ± 0.278 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9ac0fd6fd0>
├── 'a' --> tensor([[ 0.4079,  0.2904, -0.9835],
│                   [ 0.7797, -1.0047,  0.0436]])
└── 'x' --> <FastTreeValue 0x7f9ac0fd6fa0>
    └── 'c' --> tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                        [-1.0374,  0.9894,  0.7427, -0.7533],
                        [-1.7383,  0.4137,  0.2345,  0.6079]])

In [7]:
%timeit t.a = new_value

83.2 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9751,  0.0312, -0.8180],
               [-0.2948,  0.8449,  0.5180]]),
    x: Batch(
           c: tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9751,  0.0312, -0.8180],
        [-0.2948,  0.8449,  0.5180]])

In [11]:
%timeit b.a

69.5 ns ± 0.979 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5078,  1.7336, -1.1103],
               [-0.5292,  0.1546,  0.5909]]),
    x: Batch(
           c: tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079]]),
       ),
)

In [13]:
%timeit b.a = new_value

611 ns ± 5.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

169 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 837 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9a2017e8b0>
├── 'a' --> tensor([[[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]],
│           
│                   [[ 0.9751,  0.0312, -0.8180],
│                    [-0.2948,  0.8449,  0.5180]]])
└── 'x' --> <FastTreeValue 0x7f9a2017e430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

35.9 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9ac9048a30>
├── 'a' --> tensor([[ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180],
│                   [ 0.9751,  0.0312, -0.8180],
│                   [-0.2948,  0.8449,  0.5180]])
└── 'x' --> <FastTreeValue 0x7f9ac9048d30>
    └── 'c' --> tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                        [-1.0374,  0.9894,  0.7427, -0.7533],
                 

In [23]:
%timeit t_cat(trees)

33.9 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.6 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2618,  0.9832,  0.3218, -1.9537],
                       [-1.0374,  0.9894,  0.7427, -0.7533],
                       [-1.7383,  0.4137,  0.2345,  0.6079]],
              
                      [[ 0.2618,  0.9832,  0.3218, -1.9537],
                       [-1.0374,  0.9894,  0.7427, -0.7533],
                       [-1.7383,  0.4137,  0.2345,  0.6079]],
              
                      [[ 0.2618,  0.9832,  0.3218, -1.9537],
                       [-1.0374,  0.9894,  0.7427, -0.7533],
                       [-1.7383,  0.4137,  0.2345,  0.6079]],
              
                      [[ 0.2618,  0.9832,  0.3218, -1.9537],
                       [-1.0374,  0.9894,  0.7427, -0.7533],
                       [-1.7383,  0.4137,  0.2345,  0.6079]],
              
                      [[ 0.2618,  0.9832,  0.3218, -1.9537],
                       [-1.0374,  0.9894,  0.7427, -0.7533],
                       [-1.7383,  0.4137,  0.2345,  0.6079]],

In [26]:
%timeit Batch.stack(batches)

89.1 µs ± 802 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079],
                      [ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079],
                      [ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079],
                      [ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079],
                      [ 0.2618,  0.9832,  0.3218, -1.9537],
                      [-1.0374,  0.9894,  0.7427, -0.7533],
                      [-1.7383,  0.4137,  0.2345,  0.6079],
                      [ 0.2618,  0.9832,  0.3218, -1.9537],
                   

In [28]:
%timeit Batch.cat(batches)

176 µs ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

373 µs ± 5.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
